In [1]:
# -*- coding: utf-8 -*-
import numpy as np
import math, random
#import copy as cp
import time

In [46]:
side = 20
size = side*side
init_nmols = 300
ntrials = side**4 # Es buena idea que vaya como n*m
nsamples = size
j_ising = -1.0
t_env = 2.0
μ = 0.0
t_env = 3.5
zz = math.exp(μ/t_env)/math.sqrt(t_env**3)

In [3]:
def change_spin():
    return random.choice([-1, 1])

In [4]:
def initialize(side, init_nmols):
    spins = []
    neighbors = []
    occupancy = []
    
    init_mols = random.sample(range(side*side), init_nmols)
    for jj in range(side):
        for ii in range(side):
            index = jj*side + ii
            spins.append(change_spin())
            all_nbs = set([((jj + col)%side)*side + (ii + row)%side 
                for col in range(-1, 2) for row in range(-1, 2)])
            diag_nbs = set([((jj + col)%side)*side + (ii + row)%side
                for col in [-1, 1] for row in [-1, 1]])
            neighbors.append(list(all_nbs - diag_nbs - {index}))
            
            occupancy.append(index in init_mols)
    return spins, neighbors, occupancy

In [5]:
def add_spin(occupancy):
    free_sites = [ii for ii, occ in enumerate(occupancy) if not occ]
    if len(free_sites) == 0: # Si todos los sitios están ocupados
        return -1
    trial_idx = random.choice(free_sites)
    return trial_idx

In [27]:
def delete_spin(occupancy):
    occupied_sites = [ii for ii, occ in enumerate(occupancy) if occ]
    if len(occupied_sites) == 0: # Si todos los sitios están vacíos
        return -1
    trial_idx = random.choice(occupied_sites)
    return trial_idx

In [9]:
def move_spin(occupancy, size):
    occupied_sites = [ii for ii, occ in enumerate(occupancy) if occ]
    # Si todos los sitios están  ocupados o  vacíos, no se puede realizar nada
    if len(occupied_sites) in [size, 0]:
        return -1, -1
    old_idx = random.choice(occupied_sites)
    free_sites = [ii for ii, occ in enumerate(occupancy) if not occ]
    trial_idx = random.choice(free_sites)
    return old_idx, trial_idx


In [29]:
def montecarlo_cycle(size, spins, neighbors, occupancy, ntrials, nsamples, j_ising, t_env, zz):
    nerrors = 0
    energies = []
    magnets = []
    
    te = 0.0
    for jj in range(size):
        nbs = neighbors[jj]
        neighbors_spins = [spins[nb]*occupancy[nb] for nb in nbs]
        #neighbors_spins = spins[neighbors[jj]] #Lista de los espines vecinos
        te = te + potential_energy(spins[jj]*occupancy[jj], neighbors_spins, j_ising) 
    te = te/2.0
    energies.append(te)
    magnets.append(magnetization([spins[ii]*occupancy[ii] for ii in range(size)]))
    
    for ii in range(ntrials):
        #try:
        spins, occupancy, energy_change = montecarlo(size, spins, neighbors, occupancy, j_ising, t_env, zz)
        te = te + energy_change
        #except ZeroDivisionError:
        #    nerrors += 1
        #    continue    
        if ii%nsamples == 0:
            energies.append(te)
            magnets.append(magnetization([spins[ii]*occupancy[ii] for ii in range(size)]))
        
    #energies = [float("{0:.2f}".format(energies[ii])) for ii in range(len(energies))]
    
    return spins, occupancy, energies, magnets, nerrors

In [14]:
def montecarlo(size, spins, neighbors, occupancy, j_ising, t_env, zz):
    r = random.random()
    nmol = sum(occupancy)
    
    #Añadir espín
    if 0 <= r < 0.2:
        idx = add_spin(occupancy)
        # Por si no hay espacios libres para añadir otro espín
        if idx == -1:
            return spins, occupancy, 0.0
        old_energy = 0.0
        trial_spin = change_spin()
        nbs = neighbors[idx] #Es mejor así que introduciéndolo directamente a la comprensión
        neighbors_spins = [spins[nb]*occupancy[nb] for nb in nbs]        
        trial_energy = potential_energy(trial_spin, neighbors_spins, j_ising)
        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env - math.log(zz*size/(nmol + 1))):
            occupancy[idx] = True
            spins[idx] = trial_spin
            energy_change = energy_diff
            
    #Eliminar espín
    elif 0.2 <= r < 0.4:
        idx = delete_spin(occupancy)
        if idx == -1:
            return spins, occupancy, 0.0
        nbs = neighbors[idx] #Es mejor así que introduciéndolo directamente a la comprensión
        neighbors_spins = [spins[nb]*occupancy[nb] for nb in nbs]        
        old_energy = potential_energy(spins[idx], neighbors_spins, j_ising)
        trial_energy = 0.0
        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env - math.log(nmol/(zz*size))):
            occupancy[idx] = False
            energy_change = energy_diff
    
    #Mover espín a una posición vacía
    elif 0.4 <= r < 0.7:
        old_idx, trial_idx = move_spin(occupancy, size)
        # Por si no hay espacios para mover un espín
        if old_idx == -1:
            return spins, occupancy, 0.0
        old_nbs = neighbors[old_idx] #Es mejor así que introduciéndolo directamente a la comprensión
        old_neighbors_spins = [spins[nb]*occupancy[nb] for nb in old_nbs]        
        old_energy = potential_energy(spins[old_idx], old_neighbors_spins, j_ising)
        
        #Estos dos se cambian por si se mueve a un sitio vecino
        occupancy[old_idx] = False
        occupancy[trial_idx] = True
        
        trial_nbs = neighbors[trial_idx]
        trial_neighbors_spins = [spins[nb]*occupancy[nb] for nb in trial_nbs]
        trial_energy = potential_energy(spins[old_idx], trial_neighbors_spins, j_ising)
        
        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env):
            #occupancy[old_idx] = false
            spins[trial_idx] = spins[old_idx]
            #occupancy[trial_idx] = true
            energy_change = energy_diff
        else:
            # Si se intentó mover a un sitio vecino y no se aceptó el movimiento,
            # se regresan a sus valores originales
            occupancy[old_idx] = True
            occupancy[trial_idx] = False
            
    else:
        idx = delete_spin(occupancy) #Porque encuentra los sitios ocupados
        if idx == -1:
            return spins, occupancy, 0.0
        
        nbs = neighbors[idx] #Es mejor así que introduciéndolo directamente a la comprensión
        neighbors_spins = [spins[nb]*occupancy[nb] for nb in nbs]        
    
        old_energy = potential_energy(spins[idx], neighbors_spins, j_ising)
        trial_spin = -spins[idx]
        trial_energy = potential_energy(trial_spin, neighbors_spins, j_ising)

        energy_diff = trial_energy - old_energy
        energy_change = 0.0
        if metropolis_step(energy_diff/t_env):
            spins[idx] = trial_spin
            energy_change = energy_diff
    
    return spins, occupancy, energy_change

In [15]:
def potential_energy(central_spin, neighbors_spins, j_ising):
    return j_ising*central_spin*sum(neighbors_spins)

In [16]:
def magnetization(spins):
    return sum(spins)

In [17]:
def metropolis_step(exp_arg):
    """
    Returns True if the trial configuration goes towards a region with higher probability
    or it is given the chance to explore regions with fewer probability
    """
    if exp_arg < 0.0: # Energía: w < 0
        return True # Se actualiza el estado del sistema
    else:
        w = math.exp(-exp_arg)
        if random.random() < w: # Energía: w
            return True # También se actualiza el estado del sistema
        else:
            return False

### Lo que sigue ya podría ser el programa principal

In [47]:
spins, neighbors, occupancy = initialize(side, init_nmols)

In [48]:
occupancy

[False,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 Tr

In [49]:
#spins, energy_change = montecarlo(size, spins, neighbors, j_ising, t_env)

In [50]:
t1 = time.time()
spins, occupancy, energies, magnets, nerrors = montecarlo_cycle(size, spins, neighbors, occupancy, ntrials, nsamples, j_ising, t_env, zz)
t2 = time.time()
print(t2-t1)

8.184388399124146


In [51]:
#7.233415842056274 s con 20x20  y side^4
#0.674243 seconds (909.41 k allocations: 463.976 MB, 16.19% gc time) con Julia 
#y apuesto a que con numpy no mejorará

In [52]:
[mm/size for mm in magnets]

[-0.015,
 -0.0175,
 -0.0225,
 -0.0125,
 0.015,
 -0.04,
 0.015,
 0.0375,
 -0.01,
 -0.01,
 -0.0325,
 -0.0275,
 0.0225,
 -0.02,
 -0.015,
 0.0,
 -0.0175,
 0.0075,
 0.0075,
 -0.0175,
 -0.04,
 -0.0075,
 0.0025,
 0.025,
 0.005,
 -0.0325,
 -0.025,
 -0.0025,
 0.015,
 -0.005,
 -0.0225,
 -0.0175,
 -0.06,
 -0.0225,
 -0.0025,
 -0.0225,
 -0.0025,
 -0.0025,
 -0.0175,
 -0.0225,
 -0.0125,
 0.03,
 0.0075,
 -0.0075,
 0.0025,
 -0.0075,
 -0.0125,
 -0.0325,
 -0.0075,
 -0.0025,
 0.035,
 0.0075,
 0.0225,
 -0.02,
 -0.025,
 -0.01,
 0.005,
 0.035,
 0.0175,
 0.015,
 -0.005,
 0.015,
 0.01,
 -0.0175,
 -0.0075,
 -0.05,
 -0.01,
 -0.0125,
 0.04,
 -0.0225,
 -0.0025,
 0.05,
 0.0225,
 -0.0225,
 -0.0175,
 0.025,
 -0.04,
 -0.015,
 0.01,
 -0.0075,
 -0.005,
 0.0025,
 -0.0025,
 -0.02,
 0.0175,
 -0.0075,
 0.0,
 -0.0275,
 -0.0125,
 0.0,
 -0.0225,
 -0.025,
 0.015,
 -0.01,
 0.045,
 0.0025,
 0.0,
 -0.03,
 0.0175,
 0.03,
 0.0325,
 -0.0225,
 -0.03,
 0.015,
 0.005,
 0.0175,
 0.0325,
 -0.0075,
 0.0125,
 0.0225,
 0.0,
 -0.0325,
 -0.01,

In [53]:
energies

[41.0,
 41.0,
 -52.0,
 -43.0,
 -32.0,
 -21.0,
 0.0,
 -12.0,
 -7.0,
 1.0,
 -2.0,
 -11.0,
 2.0,
 -9.0,
 -3.0,
 -12.0,
 -3.0,
 -8.0,
 -6.0,
 -9.0,
 -8.0,
 2.0,
 -10.0,
 2.0,
 -1.0,
 -5.0,
 -1.0,
 -3.0,
 -2.0,
 -11.0,
 -6.0,
 -1.0,
 -17.0,
 -7.0,
 -10.0,
 -13.0,
 -3.0,
 -4.0,
 -9.0,
 -6.0,
 -6.0,
 -16.0,
 -2.0,
 -19.0,
 -1.0,
 -2.0,
 -3.0,
 -7.0,
 -10.0,
 -14.0,
 -2.0,
 -1.0,
 -4.0,
 -2.0,
 3.0,
 -5.0,
 -7.0,
 5.0,
 -15.0,
 0.0,
 -1.0,
 -10.0,
 0.0,
 -8.0,
 -3.0,
 -4.0,
 -4.0,
 -7.0,
 -6.0,
 -9.0,
 -3.0,
 -8.0,
 -7.0,
 -1.0,
 -5.0,
 -14.0,
 -9.0,
 0.0,
 -11.0,
 -1.0,
 -1.0,
 -5.0,
 2.0,
 0.0,
 1.0,
 -3.0,
 1.0,
 -5.0,
 -5.0,
 -7.0,
 0.0,
 -7.0,
 1.0,
 -6.0,
 -3.0,
 -3.0,
 -9.0,
 -5.0,
 -1.0,
 -5.0,
 -9.0,
 -8.0,
 -3.0,
 -2.0,
 -10.0,
 -1.0,
 -8.0,
 -2.0,
 -5.0,
 0.0,
 1.0,
 -4.0,
 -9.0,
 7.0,
 -10.0,
 -9.0,
 -4.0,
 -4.0,
 -8.0,
 -4.0,
 -7.0,
 -5.0,
 -10.0,
 -11.0,
 -11.0,
 -12.0,
 -9.0,
 -8.0,
 -5.0,
 -8.0,
 -6.0,
 -6.0,
 1.0,
 -9.0,
 -5.0,
 2.0,
 -12.0,
 -5.0,
 -7.0,
 1.0,
 -8.0,
 5.0,
 -